## Import required libraries

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
## Import historical data 

In [89]:
from help_functions import get_energy_data

energy_data = get_energy_data.get_data(
    'hourly_ec_20201107_20231107.csv')

In [92]:
energy_data["weekday"] = energy_data.index.weekday
energy_data.head(100)

,date,beginning,end,energy_consumption,weekday
date_time,,,,,
2020-11-07 00:00:00,2020-11-07,00:00:00,01:00:00,46.97475,5
2020-11-07 01:00:00,2020-11-07,01:00:00,02:00:00,45.35550,5
2020-11-07 02:00:00,2020-11-07,02:00:00,03:00:00,44.66450,5
2020-11-07 03:00:00,2020-11-07,03:00:00,04:00:00,44.76300,5
2020-11-07 04:00:00,2020-11-07,04:00:00,05:00:00,44.81150,5
...,...,...,...,...,...
2020-11-10 23:00:00,2020-11-10,23:00:00,00:00:00,52.01300,1
2020-11-11 00:00:00,2020-11-11,00:00:00,01:00:00,48.94825,2
2020-11-11 01:00:00,2020-11-11,01:00:00,02:00:00,47.61100,2


## Import data 
To-do: 
* via URL
* If date, beginning and ending columns unnecessary --> delete 

In [86]:

# import data 
energy_data = (pd.read_csv('hourly_ec_20201107_20231107.csv',
                 delimiter=';', encoding='utf-8')
               .iloc[:, 0:4]
               .dropna()
               .rename(columns={'Datum': 'date', 
                                        'Anfang': 'beginning', 
                                        'Ende': 'end', 
                                        'Gesamt (Netzlast) [MWh] Berechnete Auflösungen': 'energy_consumption'}))


# convert data type and measurement unit of energy consumption (in GWh instead of MWh)
energy_data = energy_data[~energy_data['energy_consumption'].str.contains('-')]  
energy_data['energy_consumption'] = (energy_data['energy_consumption'].str.replace('.', '', regex=False)
                                                                    .str.replace(',', '.', regex=False)
                                                                    .astype(float) / 1000)

# Set date_time as row index for time series 
energy_data['date'] = pd.to_datetime(energy_data['date'], format='%d.%m.%Y')
energy_data['beginning'] = pd.to_datetime(
    energy_data['beginning'], format='%H:%M').dt.time
energy_data['end'] = pd.to_datetime(
    energy_data['end'], format='%H:%M').dt.time
energy_data['date_time'] = energy_data.apply(
                                           lambda row: pd.Timestamp.combine(row['date'], row['beginning']), axis=1)
energy_data.set_index('date_time', inplace=True)

# check
energy_data.head(30)
energy_data.tail(30)
#energy_data.dtypes

,date,beginning,end,energy_consumption
date_time,,,,
2023-11-06 05:00:00,2023-11-06,05:00:00,06:00:00,49.12350
2023-11-06 06:00:00,2023-11-06,06:00:00,07:00:00,58.11750
2023-11-06 07:00:00,2023-11-06,07:00:00,08:00:00,63.23550
2023-11-06 08:00:00,2023-11-06,08:00:00,09:00:00,66.24100
2023-11-06 09:00:00,2023-11-06,09:00:00,10:00:00,67.21525
2023-11-06 10:00:00,2023-11-06,10:00:00,11:00:00,67.49250
2023-11-06 11:00:00,2023-11-06,11:00:00,12:00:00,68.75700
2023-11-06 12:00:00,2023-11-06,12:00:00,13:00:00,67.61500
2023-11-06 13:00:00,2023-11-06,13:00:00,14:00:00,67.16450


In [ ]:
# Unneccessary code 
# energy_data['Beginning'] = energy_data['Beginning'].str.split(
#    ':').str[0].astype(int)
# energy_data['End'] = energy_data['End'].str.split(
#    ':').str[0].astype(int)